In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
def setSeed(seed=1234):
  torch.manual_seed(seed)
  random.seed(seed)
  torch.cuda.manual_seed(seed)
  np.random.seed(seed)

In [ ]:
setSeed()

In [ ]:
!pip3 install https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.0.0/de_core_news_sm-3.0.0.tar.gz

     |████████████████████████████████| 19.3MB 6.6MB/s 
     |████████████████████████████████| 12.8MB 204kB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 460kB 50.9MB/s 
     |████████████████████████████████| 1.1MB 50.6MB/s 
     |████████████████████████████████| 9.1MB 44.0MB/s 
     |████████████████████████████████| 92kB 13.3MB/s 
     |████████████████████████████████| 122kB 59.2MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-3.0.0-cp37-none-any.whl size=19265762 sha256=6b9b82be4ba43546572b0ca0aaf20901252b49bdae75f6110bf89e4b23a4e0e3
  Stored in directory: /root/.cache/pip/wheels/cc/86/b2/adf9164914cc182477893072826925f9ccddb24a3302005979
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=e78c20e9b7d45868a42874d222e7a2b970d4d7f69191ac09983aa25b3689d0da
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfu

In [ ]:
!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz

     |████████████████████████████████| 13.7MB 223kB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-3.0.0-cp37-none-any.whl size=13704313 sha256=311316dee1ddd4ce99969fe758328ce8d8fe449c452fa5a3f717e8477b560f40
  Stored in directory: /root/.cache/pip/wheels/91/2b/a1/d83336e8dfaacbbcdfc805b2c7195dd3ea10d507396fe31cac
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5


In [ ]:
en=spacy.load('en_core_web_sm')
de=spacy.load('de_core_news_sm')

In [ ]:
def tokenize_en(text):
  return [tok.text for tok in en.tokenizer(text)]
def tokenize_de(text):
   return [tok.text for tok in de.tokenizer(text)][::-1]

In [ ]:
tokenize_en('hello my friend')

['hello', 'my', 'friend']

In [ ]:
SRC=Field(tokenize=tokenize_de,lower=True,eos_token='eos', init_token='sos')
TRG=Field(tokenize=tokenize_en,lower=True,eos_token='eos', init_token ='sos')

In [ ]:
SRC??

In [ ]:
train_data,valid_data,test_data= Multi30k.splits(fields=(SRC,TRG),exts=('.de','.en'))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 1.84MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 284kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 274kB/s]


In [ ]:
vars(train_data.examples[0])['src']

['.',
 'büsche',
 'vieler',
 'nähe',
 'der',
 'in',
 'freien',
 'im',
 'sind',
 'männer',
 'weiße',
 'junge',
 'zwei']

In [ ]:
SRC.build_vocab(train_data,min_freq=2)
TRG.build_vocab(train_data,min_freq=2)

In [ ]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device, type(device))

cuda <class 'torch.device'>


In [ ]:
train_loader,valid_loader,test_loader=BucketIterator.splits((train_data,valid_data,test_data),batch_size=128,device=device)
#valid_loader=BucketIterator(valid_data,batch_size=128,device=device)
#=BucketIterator(test_data,batch_size=128,device=device)

In [ ]:
for batch in train_loader:
 print(batch.src)
 break


tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [  4,   4,   4,  ...,   4,   4,   4],
        [290, 123, 422,  ..., 588, 332,   0],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]], device='cuda:0')


In [ ]:
class Encoder(nn.Module):
  def __init__(self,input_dim,embed_dim,hid_dim,n_layers,dropout):
    super().__init__()
    self.embedding=nn.Embedding(input_dim,embed_dim)
    self.rnn=nn.LSTM(input_size=embed_dim,hidden_size=hid_dim,num_layers=n_layers,dropout=dropout)
    self.dropout=nn.Dropout(dropout)
  def forward(self,src):
    embed=self.dropout(self.embedding(src))
    output,(hidden,cell)=self.rnn(embed)
    return hidden,cell


In [ ]:
class Decoder(nn.Module):
  def __init__(self,output_dim,embed_dim,hid_dim,n_layers,dropout):
    super().__init__()
    self.embedding=nn.Embedding(output_dim,embed_dim)
    self.rnn=nn.LSTM(input_size=embed_dim,hidden_size=hid_dim,num_layers=n_layers,dropout=dropout)
    self.dropout=nn.Dropout(dropout)
    self.fc=nn.Linear(hid_dim,output_dim)
    self.output_dim=output_dim
  def forward(self,src,hidden,cell):
    src=src.unsqueeze(0)
    embed=self.dropout(self.embedding(src))
    output,(hidden,cell)=self.rnn(embed,(hidden,cell))
    prediction=self.fc(output.squeeze(0))
    return prediction,hidden,cell
  

    

In [ ]:
class seq2seq(nn.Module):
  def __init__(self ,encoder,decoder,device):
    super(seq2seq,self).__init__()
    self.encoder=encoder
    self.decoder=decoder
    self.device=device
  def forward(self,src,trg,teacher_force=0.5):
    
    seqlen=trg.shape[0]
    batchsize=src.shape[1]
    vocabsize=self.decoder.output_dim
    predictions=torch.zeros((seqlen,batchsize,vocabsize),device=device)
    
    hidden,cell=self.encoder(src)
    pred,hidden,cell=self.decoder(trg[0],hidden,cell)
   
    for i in range(1,seqlen):
      
      predictions[i,:,:]=pred
      if random.random() < teacher_force:
        src=trg[i]
      else:
        src=pred.argmax(1)
      pred,hidden,cell=self.decoder(src,hidden,cell)
    return predictions

In [ ]:
input_dim,output_dim=len(SRC.vocab),len(TRG.vocab)
hid_dim=512
embed_dim=256
n_layers=2
dropout=0.5
encoder=Encoder(input_dim,embed_dim,hid_dim,n_layers,dropout)
decoder=Decoder(output_dim,embed_dim,hid_dim,n_layers,dropout)
model=seq2seq(encoder,decoder,device).to(device=device)

optimizor=optim.Adam(model.parameters())
pad_index=TRG.vocab.stoi[TRG.pad_token]
criterion=nn.CrossEntropyLoss(ignore_index=pad_index)

In [ ]:
def init_weight(m):
  for name,param in m.named_parameters():
   nn.init.uniform(param.data,-0.08,0.08)
model.apply(init_weight)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  This is separate from the ipykernel package so we can avoid doing imports until


seq2seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
    (fc): Linear(in_features=512, out_features=5893, bias=True)
  )
)

In [ ]:
train_loader

In [ ]:
def train(model,iterator,optimizor,criterion,clip=1):
  total_loss=0
  model.train()
  for i , batch in enumerate(iterator):
    
    optimizor.zero_grad()
    pred=model(batch.src,batch.trg)
    output_dim=pred.shape[-1]
    pred,trg=pred[1:].view(-1,output_dim),batch.trg[1:].view(-1)
    loss=criterion(pred,trg)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
    optimizor.step()
    #print(loss.item())
    total_loss+=loss
  return (total_loss.item())/len(iterator),torch.exp(total_loss.item()/len(iterator))


In [ ]:

loss,prep = train(model,train_loader,optimizor,criterion,)

In [ ]:
print(loss, prep)

3.969642135015143 52.96557292373904


In [ ]:
np.exp(loss)

64.63932510357404

In [ ]:
def evaluate(model,iterator,optimizor,criterion):
 
  model.eval()
  total_loss=0
  for i , batch in enumerate(iterator):
    with torch.no_grad():
      pred=model(batch.src,batch.trg)
      output_dim=pred.shape[-1]
      pred,trg=pred[1:].view(-1,output_dim),batch.trg[1:].view(-1)
      #output=pred.argmax(1)
     
      loss=criterion(pred,trg)
    
      total_loss+=loss

  return (total_loss.item())/len(iterator),math.exp(total_loss.item()/len(iterator))




In [ ]:
epochs=5
best_loss=float('inf')
for epoch in range(epochs):
  loss_train,prep_train=train(model,train_loader,optimizor,criterion,)
  loss_valid,prep_valid=evaluate(model,valid_loader,optimizor,criterion,)
  if loss_valid<best_loss:
    best_loss=loss_valid
    torch.save(model.state_dict(),'seq2seq.pt')
  print(f'\tTrain Loss: {loss_train:.3f} | Train PPL: {math.exp(loss_train):7.3f}')
  print(f'\t Val. Loss: {loss_valid:.3f} |  Val. PPL: {math.exp(loss_valid):7.3f}')

	Train Loss: 3.117 | Train PPL:  22.586
	 Val. Loss: 3.058 |  Val. PPL:  21.278
	Train Loss: 3.006 | Train PPL:  20.206
	 Val. Loss: 3.084 |  Val. PPL:  21.843
	Train Loss: 2.912 | Train PPL:  18.397
	 Val. Loss: 3.013 |  Val. PPL:  20.346
	Train Loss: 2.844 | Train PPL:  17.193
	 Val. Loss: 2.948 |  Val. PPL:  19.067
	Train Loss: 2.747 | Train PPL:  15.603
	 Val. Loss: 2.990 |  Val. PPL:  19.881


In [ ]:

model.load_state_dict(torch.load('seq2seq.pt'))

test_loss = evaluate(model, test_loader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')